In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Any, List
import chromadb
from chromadb.utils import embedding_functions

In [2]:
class DBState(TypedDict):
    query: str
    stored_id: str
    results: List[str]

In [3]:
client = chromadb.Client(
    chromadb.config.Settings(
        persist_directory="chroma_store"
    )
)

embedding_fn = embedding_functions.DefaultEmbeddingFunction()

collection = client.get_or_create_collection(
    name="demo_collection",
    embedding_function=embedding_fn
)


In [4]:

# (A) Store user query inside Chroma
def store_to_chroma(state: DBState):
    doc_id = f"id_{state['query']}"
    collection.add(
        ids=[doc_id],
        documents=[state["query"]],
        metadatas=[{"source": "user_input"}]
    )
    return {"stored_id": doc_id}


# (B) Search from Chroma
def search_chroma(state: DBState):
    result = collection.query(
        query_texts=[state["query"]],
        n_results=3
    )
    docs = result["documents"][0] if result["documents"] else []
    return {"results": docs}

In [5]:
graph = StateGraph(DBState)

graph.add_node("store", store_to_chroma)
graph.add_node("search", search_chroma)

graph.set_entry_point("store")
graph.add_edge("store", "search")
graph.add_edge("search", END)

app = graph.compile()

In [7]:
if __name__ == "__main__":
    while True:
        text = input("\nEnter something to store + search (or type 'exit' to stop): ")

        if text.lower() == "exit":
            print("Exiting workflow. Goodbye!")
            break

        output = app.invoke({"query": text})

        print("\nStored ID:", output["stored_id"])
        print("Search Results:", output["results"])



Stored ID: id_vector db
Search Results: ['vector db', 'vector database', 'search:vector']

Stored ID: id_vector
Search Results: ['vector', 'search:vector', 'vector db']

Stored ID: id_machine learning
Search Results: ['machine learning', 'vector database', 'search:vector']

Stored ID: id_vector
Search Results: ['vector', 'search:vector', 'vector db']

Stored ID: id_machine learning
Search Results: ['machine learning', 'vector database', 'search:vector']
Exiting workflow. Goodbye!
